In [1]:
import pandas as pd
import cfscrape
from lxml import etree

In [2]:
keywords = ['DDoS attack', 'denial of service','DDoS']

In [3]:
scraper = cfscrape.create_scraper()

header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.2 Safari/605.1.15'}

In [ ]:
# df = pd.DataFrame()
df= pd.read_csv('20190925_acm_dl_ddos_papers.csv')

In [4]:
for keyword in keywords:
    page = 1 
    while True:
        print(page,keyword)
        
        keyword = keyword.replace(' ','+')
        url = "https://dl.acm.org/results.cfm?query=%22"+keyword+"%22&start="+str((page-1)*20)+"&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore"
        print(url)
        
        scraped_html=scraper.get(url,headers=header).content
        html = etree.HTML(scraped_html)
        
        max_entries = html.xpath("//div[@class='pagerange']/text()")[0].split()[-1]
        if (page-1)*20 > int(max_entries):
            break
        
        df_temp = pd.DataFrame()

        #Crawler first level
        df_temp['title'] = html.xpath("//div[@class='title']/a/text()")
        df_temp['links'] = html.xpath("//div[@class='title']/a/@href")
        df_temp['links'] = (df_temp['links']+'&preflayout=flat').str.replace('citation.cfm','https://dl.acm.org/citation.cfm')
        df_temp['publication_date'] = html.xpath("//div[@class='source']/span[@class='publicationDate']/text()")
        df_temp['venue']=html.xpath("//div[@class='source']/span[@style='padding-left:10px']/text()")

        #Crawler second level
        for link in df_temp['links']:
            print(link)
            scraped_html=scraper.get(link,headers=header).content
            html = etree.HTML(scraped_html)

            index_current = df_temp.index[df_temp['links'] == link][0]

            try:
                df_temp.at[index_current, 'link4download'] = "https://dl.acm.org/" + html.xpath("//table[@class='medium-text']/tr/td/a/@href")[0]
            except:
                df_temp.at[index_current, 'link4download'] = ""

            try:
                df_temp.at[index_current, 'num_citations'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Citation' in x][0].replace('\n','').replace('·\xa0','').replace('Citation Count: ','')
            except:
                df_temp.at[index_current, 'num_citations'] =""

            try:
                df_temp.at[index_current, 'num_downloads_cu'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Downloads (cumulative)' in x][0].replace('\n','').replace('·\xa0','').replace('Downloads (cumulative): ','')
            except:
                df_temp.at[index_current, 'num_downloads_cu'] =""

            try:
                df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/p/text()")[0]
            except:
                try: 
                    df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/text()")[0]
                except:
                    try:
                        df_temp.at[index_current, 'abstract'] = html.xpath("//div[@class='flatbody']/div/text()")[0].replace('\n','')
                    except:
                        df_temp.at[index_current, 'abstract'] = ""
        
        df = df.append(df_temp,ignore_index=True)
        display(df.tail(1))
        
        page = page + 1

1 DDoS attack
https://dl.acm.org/results.cfm?query=%22DDoS+attack%22&start=0&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=2662382&preflayout=flat
https://dl.acm.org/citation.cfm?id=3103164&preflayout=flat
https://dl.acm.org/citation.cfm?id=3155161&preflayout=flat
https://dl.acm.org/citation.cfm?id=2491225&preflayout=flat
https://dl.acm.org/citation.cfm?id=2079334&preflayout=flat
https://dl.acm.org/citation.cfm?id=3274717&preflayout=flat
https://dl.acm.org/citation.cfm?id=3342037&preflayout=flat
https://dl.acm.org/citation.cfm?id=3139946&preflayout=flat
https://dl.acm.org/citation.cfm?id=2813633&preflayout=flat
https://dl.acm.org/citation.cfm?id=3287940&preflayout=flat
https://dl.acm.org/citation.cfm?id=3022260&preflayout=flat
https://dl.acm.org/citation.cfm?id=1146894&preflayout=flat
https://dl.acm.org/citation.cfm?id=3185008&preflayout=flat
https://dl.acm.org/citation.cfm?id=2070445&preflayout=flat
https://dl.acm.org/citation.cfm?i

,title,links,publication_date,venue,link4download,num_citations,num_downloads_cu,abstract
19,When the Dike Breaks: Dissecting DNS Defenses ...,https://dl.acm.org/citation.cfm?id=3278534&pre...,October 2018,IMC '18: Proceedings of the Internet Measureme...,https://dl.acm.org/ft_gateway.cfm?id=3278534&f...,0,209,The Internet's Domain Name System (DNS) is a f...


2 DDoS+attack
https://dl.acm.org/results.cfm?query=%22DDoS+attack%22&start=20&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=3021477&preflayout=flat
https://dl.acm.org/citation.cfm?id=2382255&preflayout=flat
https://dl.acm.org/citation.cfm?id=1189745&preflayout=flat
https://dl.acm.org/citation.cfm?id=3278534&preflayout=flat
https://dl.acm.org/citation.cfm?id=2381758&preflayout=flat
https://dl.acm.org/citation.cfm?id=2980289&preflayout=flat
https://dl.acm.org/citation.cfm?id=2980379&preflayout=flat
https://dl.acm.org/citation.cfm?id=2897866&preflayout=flat
https://dl.acm.org/citation.cfm?id=1241836&preflayout=flat
https://dl.acm.org/citation.cfm?id=3022336&preflayout=flat
https://dl.acm.org/citation.cfm?id=2512523&preflayout=flat
https://dl.acm.org/citation.cfm?id=2433590&preflayout=flat
https://dl.acm.org/citation.cfm?id=3005773&preflayout=flat
https://dl.acm.org/citation.cfm?id=3328789&preflayout=flat
https://dl.acm.org/citation.cfm?

,title,links,publication_date,venue,link4download,num_citations,num_downloads_cu,abstract
39,Delving Into Internet DDoS Attacks by Botnets:...,https://dl.acm.org/citation.cfm?id=3311552&pre...,December 2018,IEEE/ACM Transactions on Networking (TON): Vol...,https://dl.acm.org/ft_gateway.cfm?id=3311552&f...,0,186,Internet distributed denial of service DDoS at...


3 DDoS+attack
https://dl.acm.org/results.cfm?query=%22DDoS+attack%22&start=40&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=3281138&preflayout=flat
https://dl.acm.org/citation.cfm?id=3268968&preflayout=flat
https://dl.acm.org/citation.cfm?id=2108707&preflayout=flat
https://dl.acm.org/citation.cfm?id=2818684&preflayout=flat
https://dl.acm.org/citation.cfm?id=948114&preflayout=flat
https://dl.acm.org/citation.cfm?id=3201584&preflayout=flat
https://dl.acm.org/citation.cfm?id=2693632&preflayout=flat
https://dl.acm.org/citation.cfm?id=1229331&preflayout=flat
https://dl.acm.org/citation.cfm?id=3310175&preflayout=flat
https://dl.acm.org/citation.cfm?id=2428675&preflayout=flat
https://dl.acm.org/citation.cfm?id=2995279&preflayout=flat
https://dl.acm.org/citation.cfm?id=2393249&preflayout=flat
https://dl.acm.org/citation.cfm?id=1456240&preflayout=flat
https://dl.acm.org/citation.cfm?id=1526896&preflayout=flat
https://dl.acm.org/citation.cfm?i

...

,title,links,publication_date,venue,link4download,num_citations,num_downloads_cu,abstract
534,Microarchitectural denial of service: insuring...,https://dl.acm.org/citation.cfm?id=774905&pref...,November 2002,MICRO 35: Proceedings of the 35th annual ACM/I...,https://dl.acm.org/ft_gateway.cfm?id=774905&ft...,11,491,Simultaneous multithreading seeks to improve t...


15 denial+of+service
https://dl.acm.org/results.cfm?query=%22denial+of+service%22&start=280&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore
https://dl.acm.org/citation.cfm?id=972382&preflayout=flat
https://dl.acm.org/citation.cfm?id=1029551&preflayout=flat
https://dl.acm.org/citation.cfm?id=3149205&preflayout=flat
https://dl.acm.org/citation.cfm?id=973385&preflayout=flat
https://dl.acm.org/citation.cfm?id=3132065&preflayout=flat
https://dl.acm.org/citation.cfm?id=1023741&preflayout=flat
https://dl.acm.org/citation.cfm?id=2507310&preflayout=flat
https://dl.acm.org/citation.cfm?id=2147710&preflayout=flat
https://dl.acm.org/citation.cfm?id=944600&preflayout=flat
https://dl.acm.org/citation.cfm?id=2980379&preflayout=flat
https://dl.acm.org/citation.cfm?id=1582403&preflayout=flat
https://dl.acm.org/citation.cfm?id=1167293&preflayout=flat
https://dl.acm.org/citation.cfm?id=2043113&preflayout=flat
https://dl.acm.org/citation.cfm?id=1377489&preflayout=flat
https://dl.acm.org/ci

,title,links,publication_date,venue,link4download,num_citations,num_downloads_cu,abstract
554,Launching denial-of-service jamming attacks in...,https://dl.acm.org/citation.cfm?id=2076581&pre...,December 2011,WUWNet '11: Proceedings of the Sixth ACM Inter...,https://dl.acm.org/ft_gateway.cfm?id=2076581&f...,3,224,Recent surges in the development of Underwater...


16 denial+of+service
https://dl.acm.org/results.cfm?query=%22denial+of+service%22&start=300&filtered=&within=owners%2Eowner%3DHOSTED&dte=&bfr=&srt=%5Fscore


ValueError: Length of values does not match length of index

In [7]:
df.to_csv('20190925_acm_dl_ddos_papers.csv')

In [8]:
df

,title,links,publication_date,venue,link4download,num_citations,num_downloads_cu,abstract
0,POSTER: How Distributed Are Today's DDoS Attacks?,https://dl.acm.org/citation.cfm?id=2662382&pre...,November 2014,CCS '14: Proceedings of the 2014 ACM SIGSAC Co...,https://dl.acm.org/ft_gateway.cfm?id=2662382&f...,1,548,Today botnets are responsible for most of the ...
1,Towards DDoS Attack Resilient Wide Area Monito...,https://dl.acm.org/citation.cfm?id=3103164&pre...,August 2017,ARES '17: Proceedings of the 12th Internationa...,https://dl.acm.org/ft_gateway.cfm?id=3103164&f...,0,88,The traditional physical power grid is evolvin...
2,Using CPR Metric to Detect and Filter Low-Rate...,https://dl.acm.org/citation.cfm?id=3155161&pre...,December 2017,SoICT 2017: Proceedings of the Eighth Internat...,,,,
3,PIT overload analysis in content centric networks,https://dl.acm.org/citation.cfm?id=2491225&pre...,August 2013,ICN '13: Proceedings of the 3rd ACM SIGCOMM wo...,https://dl.acm.org/ft_gateway.cfm?id=2491225&f...,7,223,Content Centric Networking represents a paradi...
4,Detecting DDoS attacks with Hadoop,https://dl.acm.org/citation.cfm?id=2079334&pre...,December 2011,CoNEXT '11 Student: Proceedings of The ACM CoN...,https://dl.acm.org/ft_gateway.cfm?id=2079334&f...,5,669,Recent distributed denial-of-service (DDoS) at...
5,SENSS Against Volumetric DDoS Attacks,https://dl.acm.org/citation.cfm?id=3274717&pre...,December 2018,ACSAC '18: Proceedings of the 34th Annual Comp...,https://dl.acm.org/ft_gateway.cfm?id=3274717&f...,0,125,Volumetric distributed denial-of-service (DDoS...
6,FlowJustifier: An optimized trust-based reques...,https://dl.acm.org/citation.cfm?id=3342037&pre...,July 2019,ICFNDS '19: Proceedings of the 3rd Internation...,https://dl.acm.org/ft_gateway.cfm?id=3342037&f...,0,9,Software-defined networks have introduced inno...
7,Sounding the Bell for Improving Internet (of T...,https://dl.acm.org/citation.cfm?id=3139946&pre...,November 2017,IoTS&P '17: Proceedings of the 2017 Workshop o...,,,,
8,Maneuvering Around Clouds: Bypassing Cloud-bas...,https://dl.acm.org/citation.cfm?id=2813633&pre...,October 2015,CCS '15: Proceedings of the 22nd ACM SIGSAC Co...,https://dl.acm.org/ft_gateway.cfm?id=2813633&f...,7,739,The increase of Distributed Denial-of-Service ...
9,Techniques for Improving Performance of the CP...,https://dl.acm.org/citation.cfm?id=3287940&pre...,December 2018,SoICT 2018: Proceedings of the Ninth Internati...,https://dl.acm.org/ft_gateway.cfm?id=3287940&f...,0,12,TCP-targeted low-rate distributed denial-of-se...


In [21]:
df['id'] = df['links'].apply(lambda x: x.replace("https://dl.acm.org/citation.cfm?id=","").replace("&preflayout=flat",""))

In [25]:
df['dm']=""
df['attack_type']= ""
df['defence_deployment']=""
df['not_attack_type']=""
df['attack_impact']=""
df['attack_infra']=""
df['overview']= ""

In [34]:
df[['id','abstract','overview','dm','attack_type','defence_deployment','attack_infra','attack_impact','not_attack_type']].to_csv('abstracts4classifying.csv',sep='|')

In [37]:
df.iloc[179]['abstract']

'The Internet is not secure due to its design goals being at odds\nwith the principle of least privilege. The Internet strives to\nallow any host to communicate with any other host, while the\nprinciple of least privilege advocates limiting host connectivity\nto the smallest set necessary for performing a task. Our goal is to\nsecure the Internet by largely turning off connectivity in the\nInternet, and then using explicit signaling to selectively enable\nonly those connections that are deemed necessary for performing a\ntask.'

## SENDING EMAIL AFTER THE SCRIPT ABOVE

In [ ]:
def send_email():
import smtplib
    gmail_user = 'jjsantanna.script@gmail.com'
    gmail_password = 'hosRep-tibjiw-xatca2'
    sent_from = gmail_user
    to = ['j.j.santanna@utwente.nl']
    subject = 'Script Ended'
    body = "Hey bitch, take a look on the output of your script!"
    email_text = "From: %s\nTo: %s\nSubject: %s\n\n%s\n" % (sent_from, ", ".join(to), subject, body)
try:
        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.ehlo()
        server.login(gmail_user, gmail_password)
        server.sendmail(sent_from, to, email_text)
        server.close()
print ('Email sent!')
except:
print ('Something went wrong...')

## DEBUGING ERROS 2nd LEVEL CRAWLER

In [ ]:
link = "https://dl.acm.org/citation.cfm?id=372148&preflayout=flat"

In [ ]:
scraped_html=scraper.get(link,headers=header).content
print(scraped_html)

In [ ]:
html = etree.HTML(scraped_html)

df_temp = pd.DataFrame()
index_current = 0

try:
    df_temp.at[index_current, 'link4download'] = "https://dl.acm.org/" + html.xpath("//table[@class='medium-text']/tr/td/a/@href")[0]
except:
    df_temp.at[index_current, 'link4download'] = ""

try:
    df_temp.at[index_current, 'num_citations'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Citation' in x][0].replace('\n','').replace('·\xa0','').replace('Citation Count: ','')
except:
    df_temp.at[index_current, 'num_citations'] =""

try:
    df_temp.at[index_current, 'num_downloads_cu'] = [x for x in html.xpath("//table[@class='medium-text']/tr/td[@class='small-text']/text()") if 'Downloads (cumulative)' in x][0].replace('\n','').replace('·\xa0','').replace('Downloads (cumulative): ','')
except:
    df_temp.at[index_current, 'num_downloads_cu'] =""

try:
    df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/p/text()")[0]
except:
    try: 
        df_temp.at[index_current, 'abstract'] = html.xpath("//div[@style='display:inline']/text()")[0]
    except:
        try:
            df_temp.at[index_current, 'abstract'] = html.xpath("//div[@class='flatbody']/div/text()")[0].replace('\n','')
        except:
            df_temp.at[index_current, 'abstract'] = ""

df_temp